📓 CNN PCA Graph Forecasting Notebook

Train a CNN to learn latent PCA macroeconomic trendlines from 15-year historical graphs to 2-year projected trajectories.

---

🧩 Cell 1 – Imports

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import pandas as pd

⚙️ Cell 2 – Configuration

In [5]:
# --- Config ---
IMG_HEIGHT = 128
IMG_WIDTH = 256
BATCH_SIZE = 16
DATA_ROOT = "pca_graph_pairs"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

📁 Cell 3 – Custom Dataset Loader

In [7]:
# --- Dataset ---
class GraphImageDataset(Dataset):
    def __init__(self, meta_csv, transform=None):
        self.df = pd.read_csv(meta_csv)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        input_img = Image.open(row["input_img"]).convert("RGB").resize((IMG_WIDTH, IMG_HEIGHT))
        output_img = Image.open(row["output_img"]).convert("RGB").resize((IMG_WIDTH, IMG_HEIGHT))
        if self.transform:
            input_img = self.transform(input_img)
            output_img = self.transform(output_img)
        return input_img, output_img

🎨 Cell 4 – Image Transformations

In [10]:
# --- Transforms ---
transform = transforms.Compose([
    transforms.ToTensor(),
])

🧠 Cell 6 – CNN Architecture (U-Net-style)

In [13]:
# --- Model ---
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1),  # -> (32, 64, 128)
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1), # -> (64, 32, 64)
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1),# -> (128, 16, 32)
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),  # -> (64, 32, 64)
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),   # -> (32, 64, 128)
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, 2, 1),    # -> (3, 128, 256)
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        return self.decoder(x)

🧰 Cell 7 – Initialize Model, Loss, Optimizer

In [16]:
model = SimpleCNN().to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [26]:
# --- Load Data ---
# dataset = GraphImageDataset(meta_csv=os.path.join(DATA_ROOT, "pairs_metadata.csv"), transform=transform)
# dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Load PCA graph image dataset
dataset = GraphImageDataset(
    meta_csv=os.path.join("pca_graph_pairs", "pairs_metadata.csv"),
    transform=transforms.Compose([transforms.ToTensor()])
)

# Initialize DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

print(f"✅ Loaded {len(dataset)} samples for training.")

✅ Loaded 15485 samples for training.


🏋️‍♂️ Cell 8 – Training Loop

In [28]:
# --- Training Loop ---
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in dataloader:
        x, y = batch
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss / len(dataloader):.4f}")

Epoch 1/10 - Loss: 0.0250
Epoch 2/10 - Loss: 0.0173
Epoch 3/10 - Loss: 0.0173
Epoch 4/10 - Loss: 0.0173
Epoch 5/10 - Loss: 0.0173
Epoch 6/10 - Loss: 0.0172
Epoch 7/10 - Loss: 0.0166
Epoch 8/10 - Loss: 0.0101
Epoch 9/10 - Loss: 0.0093
Epoch 10/10 - Loss: 0.0090


💾 Cell 9 – Save Trained Model

In [30]:
# --- Save model ---
torch.save(model.state_dict(), "cnn_pca_forecaster.pt")